# Форматы данных (1)

Материалы:
* Макрушин С.В. "Лекция 4: Форматы данных"
* https://docs.python.org/3/library/json.html
* https://docs.python.org/3/library/pickle.html
* https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html
* Уэс Маккини. Python и анализ данных

## Задачи для совместного разбора

1. Вывести все адреса электронной почты, содержащиеся в адресной книге `addres-book.json`

In [1]:
import json

with open("data/addres-book.json", 'r', encoding='utf8') as f:
    js = json.load(f)

[acc['email'] for acc in js] 

['faina@mail.ru', 'robert@mail.ru', 'faina@mail.ru', 'robert@mail.ru']

2. Вывести телефоны, содержащиеся в адресной книге `addres-book.json`

In [2]:
[acc['phones'] for acc in js] 

[[{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}],
 [{'phone': '232-19-55'}, {'phone': '+7 (916) 232-19-55'}],
 [{'phone': '111-19-55'}, {'phone': '+7 (916) 445-19-55'}]]

3. По данным из файла `addres-book-q.xml` сформировать список словарей с телефонами каждого из людей. 

In [3]:
from bs4 import BeautifulSoup

with open("data/addres-book-q.xml", 'r') as f:
    xml = BeautifulSoup(f, "xml")

[ {adr.find("name").text : [ph.text for ph in adr.find_all("phone")] } for adr in xml.find_all("address")]

[{'Aicha Barki': ['+ (213) 6150 4015', '+ (213) 2173 5247']},
 {'Francisco Domingos': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Maria Luisa': ['+ (244) 4232 2836']},
 {'Abraao Chanda': ['+ (244-2) 325 023', '+ (244-2) 325 023']},
 {'Beatriz Busaniche': ['+ (54-11) 4784 1159']},
 {'Francesca Beddie': ['+ (61-2) 6274 9500', '+ (61-2) 6274 9513']},
 {'Graham John Smith': ['+ (61-3) 9807 4702']}]

## Лабораторная работа №4

### JSON

1.1 Считайте файл `contributors_sample.json`. Воспользовавшись модулем `json`, преобразуйте содержимое файла в соответствующие объекты python. Выведите на экран информацию о первых 3 пользователях.

In [8]:
with open("data/contributors_sample.json", 'r', encoding='utf8') as f:
    data = json.load(f)

data[:3]

[{'username': 'uhebert',
  'name': 'Lindsey Nguyen',
  'sex': 'F',
  'address': '01261 Cameron Spring\nTaylorfurt, AK 97791',
  'mail': 'jsalazar@gmail.com',
  'jobs': ['Energy engineer',
   'Engineer, site',
   'Environmental health practitioner',
   'Biomedical scientist',
   'Jewellery designer'],
  'id': 35193},
 {'username': 'vickitaylor',
  'name': 'Cheryl Lewis',
  'sex': 'F',
  'address': '66992 Welch Brooks\nMarshallshire, ID 56004',
  'mail': 'bhudson@gmail.com',
  'jobs': ['Music therapist',
   'Volunteer coordinator',
   'Designer, interior/spatial'],
  'id': 91970},
 {'username': 'sheilaadams',
  'name': 'Julia Allen',
  'sex': 'F',
  'address': 'Unit 1632 Box 2971\nDPO AE 23297',
  'mail': 'darren44@yahoo.com',
  'jobs': ['Management consultant',
   'Engineer, structural',
   'Lecturer, higher education',
   'Theatre manager',
   'Designer, textile'],
  'id': 1848091}]

1.2 Выведите уникальные почтовые домены, содержащиеся в почтовых адресах людей

In [10]:
unique = set( [acc['mail'].split('@')[1] for acc in data] )
unique

{'gmail.com', 'hotmail.com', 'yahoo.com'}

1.3 Напишите функцию, которая по `username` ищет человека и выводит информацию о нем. Если пользователь с заданным `username` отсутствует, возбудите исключение `ValueError`

In [16]:
def getUser(username):
    search = next((acc for acc in data if acc['username'] == username), None)
    if search: return search
    else: raise ValueError
            
getUser("vickitaylor")

{'username': 'vickitaylor',
 'name': 'Cheryl Lewis',
 'sex': 'F',
 'address': '66992 Welch Brooks\nMarshallshire, ID 56004',
 'mail': 'bhudson@gmail.com',
 'jobs': ['Music therapist',
  'Volunteer coordinator',
  'Designer, interior/spatial'],
 'id': 91970}

1.4 Посчитайте, сколько мужчин и женщин присутсвует в этом наборе данных.

In [22]:
sexes = [acc['sex'] for acc in data]
{'F': sexes.count('F'), 'M':sexes.count('M')}

{'F': 2136, 'M': 2064}

1.5 Создайте `pd.DataFrame` `contributors`, имеющий столбцы `id`, `username` и `sex`.

In [27]:
import pandas as pd
contributors = pd.DataFrame(data, columns=['id', 'username', 'sex'])
contributors

,id,username,sex
0,35193,uhebert,F
1,91970,vickitaylor,F
2,1848091,sheilaadams,F
3,50969,nicole82,F
4,676820,jean67,M
...,...,...,...
4195,423555,stevenspencer,F
4196,35251,rwilliams,M
4197,135887,lmartinez,F
4198,212714,brendahill,M


1.6 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Объедините `recipes` с таблицей `contributors` с сохранением строк в том случае, если информация о человеке отсутствует в JSON-файле. Для скольких человек информация отсутствует? 

In [45]:
recipes = pd.read_csv("data/recipes_sample.csv")
recipes = recipes.merge(contributors, how="left", left_on="contributor_id", right_on='id')
print(recipes['id_y'].isna().sum())
recipes

15059


,name,id_x,minutes,contributor_id,submitted,n_steps,description,n_ingredients,id_y,username,sex
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0,35193.0,uhebert,F
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN,91970.0,vickitaylor,F
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0,NaN,NaN,NaN
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN,NaN,NaN,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16.0,this is based on a french recipe but i changed...,10.0,200862.0,ana38,F
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,NaN,"this is a traditional fresh plum cake, thought...",11.0,177443.0,douglas33,F
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,NaN,this is a traditional late summer early fall s...,NaN,NaN,NaN,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,NaN,this is a delicious soup that i originally fou...,NaN,227978.0,jessica22,M


### pickle

2.1 На основе файла `contributors_sample.json` создайте словарь следующего вида: 
```
{
    должность: [список username людей, занимавших эту должность]
}
```

In [53]:
data
# [acc['username'] for acc in data]
jobs = {}
jobs = {job: jobs.get(job, []).append(acc['username']) for acc in data for job in acc['jobs']}
jobs

{'Energy engineer': None,
 'Engineer, site': None,
 'Environmental health practitioner': None,
 'Biomedical scientist': None,
 'Jewellery designer': None,
 'Music therapist': None,
 'Volunteer coordinator': None,
 'Designer, interior/spatial': None,
 'Management consultant': None,
 'Engineer, structural': None,
 'Lecturer, higher education': None,
 'Theatre manager': None,
 'Designer, textile': None,
 'Mechanical engineer': None,
 'Retail banker': None,
 'Barrister': None,
 'Network engineer': None,
 'Youth worker': None,
 'Primary school teacher': None,
 'Engineer, broadcasting (operations)': None,
 'Designer, ceramics/pottery': None,
 'Engineer, energy': None,
 'Engineer, manufacturing': None,
 'Designer, furniture': None,
 'Sport and exercise psychologist': None,
 'Hospital pharmacist': None,
 'Surveyor, minerals': None,
 'Geochemist': None,
 'Restaurant manager': None,
 'Clinical psychologist': None,
 'Sports development officer': None,
 'Firefighter': None,
 'Charity fundraiser': 

2.2 Сохраните результаты в файл `job_people.pickle` и в файл `job_people.json` с использованием форматов pickle и JSON соответственно. Сравните объемы получившихся файлов. При сохранении в JSON укажите аргумент `indent`.

In [ ]:
with open("./data/job_people.pickle", "wb") as f:
    pickle.dump(jobLitUsers, f)

with open("./data/job_people.json", "w") as f:
    json.dump(jobLitUsers, f, indent=1)

##Файл JSON больше

: 

2.3 Считайте файл `job_people.pickle` и продемонстрируйте, что данные считались корректно. 

In [ ]:
with open("./data/job_people.pickle", "rb") as f:
    job_people = pickle.load(f)
pp(job_people)

: 

### XML

3.1 По данным файла `steps_sample.xml` сформируйте словарь с шагами по каждому рецепту вида `{id_рецепта: ["шаг1", "шаг2"]}`. Сохраните этот словарь в файл `steps_sample.json`

In [ ]:
with open("./data/steps_sample.xml", "r", encoding="UTF-8") as f:
    steps_sample = BeautifulSoup(f, "xml")

: 

In [ ]:
dict_xml = {}
for recipe in steps_sample.recipes.find_all("recipe"):
    if recipe.id.text not in dict_xml:
        dict_xml[recipe.id.text] = [] #recipe.find_all("steps")
    for steps in recipe.find_all("step"):
        dict_xml[recipe.id.text].append(steps.text)
dict_xml

: 

In [ ]:
with open("./data/steps_sample.json", "w") as f:
    json.dump(dict_xml, f, indent=1)

: 

3.2 По данным файла `steps_sample.xml` сформируйте словарь следующего вида: `кол-во_шагов_в_рецепте: [список_id_рецептов]`

In [ ]:
count_steps_recipes = {}
for recipes in steps_sample.recipes.find_all("recipe"):
    if len(recipes.steps) not in count_steps_recipes:
        count_steps_recipes[len(recipes.steps)] = []
    count_steps_recipes[len(recipes.steps)].append(recipes.find("id").text)

pp(count_steps_recipes)

: 

3.3 Получите список рецептов, в этапах выполнения которых есть информация о времени (часы или минуты). Для отбора подходящих рецептов обратите внимание на атрибуты соответствующих тэгов.

In [ ]:
time_recipes = []
for recipe in steps_sample.recipes.find_all("recipe"):
    if recipe.find("step", attrs={'has_minutes': '1'}) or recipe.find("step", attrs={'has_hours': "1"}):
        time_recipes.append(recipe.id.text)
time_recipes

: 

3.4 Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в таблицу `recipes`. Для строк, которые содержат пропуски в столбце `n_steps`, заполните этот столбец на основе файла  `steps_sample.xml`. Строки, в которых столбец `n_steps` заполнен, оставьте без изменений.

In [ ]:
recipes_sample = pd.read_csv("../../ЛР2/02_pandas/data/recipes_sample.csv", sep=",")
recipes_sample[:100]

: 

In [ ]:
for recipe in steps_sample.recipes.find_all("recipe"):
    recipe_id = int(recipe.id.text)
    if np.isnan(recipes_sample[recipes_sample["id"] == recipe_id]["n_steps"].iloc[0]):
        recipes_sample.loc[recipes_sample.id == recipe_id, "n_steps"] = len(recipe.find_all("step"))

recipes_sample[:100]

: 

3.5 Проверьте, содержит ли столбец `n_steps` пропуски. Если нет, то преобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_filled_nsteps.csv`

In [ ]:
pp(np.sum(np.isnan(recipes_sample["n_steps"])))
recipes_sample["n_steps"] = recipes_sample["n_steps"].astype("Int64")
recipes_sample.head()

: 

In [ ]:
recipes_sample.to_csv("./data/recipes_sample_with_filled_nsteps.csv")

: 